# Loss Ratio Prediction

In [63]:
import os
import math
import csv
import collections
import pdb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso

%matplotlib inline

## Load Dataset

In [64]:
path_training_dataset = "training_data.csv"
df_data = pd.read_csv(path_training_dataset)

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\4148554216.py:2: DtypeWarning: Columns (6,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(path_training_dataset)


In [65]:
df_data.head()

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,...,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,...,100-400,2006,Y,N,320.12,0,0.0,0.0,0.0,0.0
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,...,100-200,2006,Y,N,259.70,0,0.0,0.0,0.0,0.0
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,...,100-400,2006,Y,N,613.74,0,0.0,0.0,0.0,0.0
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0


### Common Functions

In [66]:
def get_selected_features(feature_list_file_path):
    with open(feature_list_file_path) as f:
        features_list = f.read().splitlines() 
        return features_list

In [67]:
def strip_white_spaces(input_df):
    columns = list(input_df.columns)
    for column in columns:
        input_df[column] = input_df[column].str.strip()
    return input_df

In [68]:
def convert_df_objects_to_str(df_dataset, df_columns):
    for column in df_columns:
        if df_dataset[column].dtype == 'object':
            df_dataset[column] = df_dataset[column].astype(str)
    return df_dataset

In [69]:
def get_num_cat_features(df_dataset, df_columns):
    categorical_variables = []
    numerical_variables = []
    for column in df_columns:
        print(column)
        if df_dataset[column].dtype == 'object':
            categorical_variables.append(column)
        else:
            numerical_variables.append(column)
    return numerical_variables, categorical_variables

In [70]:
selected_features_train = [
    'Vehicle_Anti_Theft_Device',
    'Driver_Total_Female',
    'Driver_Total_Male',
    'Driver_Total_Teenager_Age_15_19',
    'Driver_Total_College_Ages_20_23',
    'Driver_Total_Young_Adult_Ages_24_29',
    'Driver_Total_Low_Middle_Adult_Ages_30_39',
    'Driver_Total_Middle_Adult_Ages_40_49',
    'Driver_Total_Adult_Ages_50_64',
    'Driver_Total_Senior_Ages_65_69',
    'Driver_Total_Upper_Senior_Ages_70_plus',
    'Driver_Total_Married',
    'Driver_Total_Single',
    'Vehicle_Usage',
    'Vehicle_Miles_To_Work',
    'Vehicle_Territory',
    'Annual_Premium',
    'Loss_Amount']

In [71]:
df_data = df_data[selected_features_train]

In [72]:
num_features, cat_features = get_num_cat_features(df_data, list(df_data.columns))

Vehicle_Anti_Theft_Device
Driver_Total_Female
Driver_Total_Male
Driver_Total_Teenager_Age_15_19
Driver_Total_College_Ages_20_23
Driver_Total_Young_Adult_Ages_24_29
Driver_Total_Low_Middle_Adult_Ages_30_39
Driver_Total_Middle_Adult_Ages_40_49
Driver_Total_Adult_Ages_50_64
Driver_Total_Senior_Ages_65_69
Driver_Total_Upper_Senior_Ages_70_plus
Driver_Total_Married
Driver_Total_Single
Vehicle_Usage
Vehicle_Miles_To_Work
Vehicle_Territory
Annual_Premium
Loss_Amount


In [73]:
num_features

['Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount']

In [74]:
cat_features

['Vehicle_Anti_Theft_Device', 'Vehicle_Usage']

### Cleaning Training Dataset

In [75]:
df_num = df_data[num_features]
df_num.head()

,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount
0,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0
1,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0
2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0
3,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0
4,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0


In [76]:
df_cat = df_data[cat_features]
df_cat.head()

,Vehicle_Anti_Theft_Device,Vehicle_Usage
0,Not Applicable,Farm
1,Not Applicable,Farm
2,Passive Disabling-Vehicle Recovery,Pleasure
3,Passive Disabling-Vehicle Recovery,Pleasure
4,Passive Disabling-Vehicle Recovery,Pleasure


In [77]:
df_cat_strip = strip_white_spaces(df_cat)

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()


In [78]:
df_data_clean = df_num.join(df_cat, how='outer')

In [79]:
df_data_clean.head()

,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount,Vehicle_Anti_Theft_Device,Vehicle_Usage
0,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0,Not Applicable,Farm
1,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0,Not Applicable,Farm
2,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0,Passive Disabling-Vehicle Recovery,Pleasure
3,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Passive Disabling-Vehicle Recovery,Pleasure
4,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Passive Disabling-Vehicle Recovery,Pleasure


In [80]:
df_data_clean.shape

(424431, 18)

In [81]:
df_data_clean.to_csv("cleaned.csv")

### Find useful features

In [82]:
def get_unique_values_with_count(df_input):
    columns = df_input.columns
    for column in columns:
        unique_value_count = df_input[column].value_counts()
        print(column)
        print()
        print(unique_value_count)
        print('--'*50)

In [83]:
get_unique_values_with_count(df_data)

Vehicle_Anti_Theft_Device

Not Applicable                        248634
Passive Disabling-Vehicle Recovery    133362
Alarm Only                             31852
Active Disabling                       10578
Unknown                                    5
Name: Vehicle_Anti_Theft_Device, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Female

1    238676
0    181914
2      3837
3         4
Name: Driver_Total_Female, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Male

1    222967
0    198925
2      2537
3         2
Name: Driver_Total_Male, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Teenager_Age_15_19

0    407343
1     16957
2       131
Name: Driver_Total_Teenager_Age_15_19, dtype: int64
----------------------------------------------------------------

### Features to be used in portfolio

- Vehicle_Make_Year_Mean
- Vehicle_Performance_Standard_Minus_all
- Vehicle_Number_Of_Drivers_Assigned_Mean_Ignore_gt_10
- Vehicle_Usage_Pleasure_Minus_all
- Vehicle_Anti_Theft_Device_anything_but_not_applicable
- Vehicle_Age_In_Years_Mean
- Vehicle_Collision_Coverage_Indicator_Y_minus_N
- Driver_Total_Single_Sum_not_include_0
- Driver_Total_Mean
- Driver_Total_Male_Mean
- Driver_Total_Female_Mean
- Driver_Total_Married_Mean
- Driver_Total_Teenager_Age_15_19_Mean
- Driver_Total_College_Ages_20_23_Sum
- Driver_Total_Young_Adult_Ages_24_29_Mean
- Driver_Total_Upper_Senior_Ages_70_plus_Mean
- Annual_Premium_Sum

In [84]:
selected_features_test = ['Vehicle_Make_Year',
                        'Vehicle_Number_Of_Drivers_Assigned',
                        'Vehicle_Age_In_Years',
                        'Driver_Total_Single',
                        'Driver_Total',
                        'Driver_Total_Male',
                        'Driver_Total_Female',
                        'Driver_Total_Married',
                        'Driver_Total_Teenager_Age_15_19',
                        'Driver_Total_College_Ages_20_23',
                        'Driver_Total_Young_Adult_Ages_24_29',
                        'Driver_Total_Upper_Senior_Ages_70_plus',
                        'Vehicle_Usage',
                        'Vehicle_Anti_Theft_Device',
                        'Vehicle_Performance',
                        'Annual_Premium']

### Create Portfolios in training data like testing portfolios

In [85]:
df_data.shape

(424431, 18)

In [86]:
policies_in_portfolios = [1000, 3000, 5000]
percentage_losses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]

In [87]:
df_list_portfolios = []

In [88]:
df_data_oh = pd.get_dummies(df_data)

In [89]:
df_data_oh.shape

(424431, 25)

In [90]:
df_data_oh_columns = list(df_data_oh.columns)

In [91]:
for i in range(10):
    df_data = df_data_oh.sample(frac=1)
    df_data_loss_zero = df_data[df_data['Loss_Amount'] == 0]
    df_data_loss_non_zero = df_data[df_data['Loss_Amount'] != 0]
    current_df_without_loss = 0
    current_df_with_loss = 0
    for i in range(len(policies_in_portfolios)):
        for j in range(len(percentage_losses)):
            without_loss_size = int(((100 - percentage_losses[j])/100) * policies_in_portfolios[i])
            with_loss_size = int(policies_in_portfolios[i] - without_loss_size )

            df_without_loss = df_data_loss_zero.iloc[current_df_without_loss: current_df_without_loss + without_loss_size]
            df_with_loss = df_data_loss_non_zero.iloc[current_df_with_loss: current_df_with_loss + with_loss_size]

            current_df_without_loss = current_df_without_loss + without_loss_size
            current_df_with_loss = current_df_with_loss + with_loss_size

            df_merge = pd.concat([df_without_loss, df_with_loss])
            df_list_portfolios.append(df_merge)

In [92]:
len(df_list_portfolios)

330

In [93]:
df_list_portfolios

[        Driver_Total_Female  Driver_Total_Male  \
 384608                    0                  1   
 210223                    0                  1   
 396578                    0                  1   
 399181                    1                  0   
 327871                    1                  0   
 ...                     ...                ...   
 107941                    1                  0   
 255682                    1                  1   
 386170                    1                  0   
 418916                    1                  0   
 326320                    0                  2   
 
         Driver_Total_Teenager_Age_15_19  Driver_Total_College_Ages_20_23  \
 384608                                0                                0   
 210223                                0                                0   
 396578                                0                                0   
 399181                                0                                0   
 

In [94]:
count = 0 
for i in range(len(df_list_portfolios)):
    temp = df_list_portfolios[i]
    count = count + temp.shape[0]
print(count)

990000


In [95]:
df_list_portfolios[0]

,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,...,Loss_Amount,Vehicle_Anti_Theft_Device_Active Disabling,Vehicle_Anti_Theft_Device_Alarm Only,Vehicle_Anti_Theft_Device_Not Applicable,Vehicle_Anti_Theft_Device_Passive Disabling-Vehicle Recovery,Vehicle_Anti_Theft_Device_Unknown,Vehicle_Usage_Business,Vehicle_Usage_Farm,Vehicle_Usage_Pleasure,Vehicle_Usage_Work
384608,0,1,0,0,1,0,0,0,0,0,...,0.00,0,0,1,0,0,0,1,0,0
210223,0,1,0,0,0,0,0,1,0,0,...,0.00,1,0,0,0,0,0,0,1,0
396578,0,1,0,0,0,0,0,1,0,0,...,0.00,0,0,1,0,0,0,1,0,0
399181,1,0,0,0,0,0,0,1,0,0,...,0.00,0,0,0,1,0,0,0,1,0
327871,1,0,0,0,0,0,0,1,0,0,...,0.00,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107941,1,0,0,0,0,0,0,1,0,0,...,54.50,0,0,0,1,0,0,0,1,0
255682,1,1,0,0,0,0,1,1,0,0,...,648.55,0,0,1,0,0,0,1,0,0
386170,1,0,0,0,0,0,0,1,0,0,...,161.67,0,0,1,0,0,0,0,1,0
418916,1,0,0,0,0,1,0,0,0,0,...,109.00,0,0,0,1,0,0,0,0,1


In [96]:
df_data_final = None

In [97]:
df_data_oh_columns

['Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount',
 'Vehicle_Anti_Theft_Device_Active Disabling',
 'Vehicle_Anti_Theft_Device_Alarm Only',
 'Vehicle_Anti_Theft_Device_Not Applicable',
 'Vehicle_Anti_Theft_Device_Passive Disabling-Vehicle Recovery',
 'Vehicle_Anti_Theft_Device_Unknown',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [98]:
selected_features_mean = num_features.copy()
selected_features_mean.remove('Annual_Premium')
selected_features_mean.remove('Loss_Amount')
mean_columns = selected_features_mean

sum_columns = ['Annual_Premium']

for cat_f in cat_features:
    for col in list(df_list_portfolios[0].columns):
        if col.startswith(cat_f):
            sum_columns.append(col)

target_column = ['Loss_Amount']

In [99]:
sum_columns

['Annual_Premium',
 'Vehicle_Anti_Theft_Device_Active Disabling',
 'Vehicle_Anti_Theft_Device_Alarm Only',
 'Vehicle_Anti_Theft_Device_Not Applicable',
 'Vehicle_Anti_Theft_Device_Passive Disabling-Vehicle Recovery',
 'Vehicle_Anti_Theft_Device_Unknown',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [100]:
ls_final_training = []

In [101]:
mean_columns

['Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory']

In [102]:
target_column

['Loss_Amount']

In [103]:
def portfolio_to_features(df_portfolio, is_training=True):
    df_portfolio_mean = df_portfolio[mean_columns]
    df_portfolio_mean = df_portfolio_mean.mean()
    ls_portfolio_mean = df_portfolio_mean.to_list()
    
    df_portfolio_sum = df_portfolio[sum_columns]
    df_portfolio_sum = df_portfolio_sum.sum()
    ls_portfolio_sum = df_portfolio_sum.to_list()
    
    if is_training:
        df_portfolio_target = df_portfolio[target_column]
        df_portfolio_target_sum = df_portfolio_target.sum()
        ls_portfolio_target_sum = df_portfolio_target_sum.to_list()
    
    list_merge = ls_portfolio_mean + ls_portfolio_sum
    
    if is_training:
        list_merge = list_merge + ls_portfolio_target_sum
    
    return list_merge

In [104]:
def features_to_training_df(df_portfolios_list):
    features_list = mean_columns + sum_columns + target_column
    final_list = []
    for single_portfolio in df_portfolios_list:
        ls_portfolio = portfolio_to_features(single_portfolio)
        final_list.append(ls_portfolio)
    df_final_training = pd.DataFrame(final_list, columns = features_list)
    return df_final_training

In [105]:
df_final_train = features_to_training_df(df_list_portfolios)

In [106]:
df_final_train.head()

,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,...,Vehicle_Anti_Theft_Device_Active Disabling,Vehicle_Anti_Theft_Device_Alarm Only,Vehicle_Anti_Theft_Device_Not Applicable,Vehicle_Anti_Theft_Device_Passive Disabling-Vehicle Recovery,Vehicle_Anti_Theft_Device_Unknown,Vehicle_Usage_Business,Vehicle_Usage_Farm,Vehicle_Usage_Pleasure,Vehicle_Usage_Work,Loss_Amount
0,0.574,0.538,0.043,0.030,0.086,0.220,0.258,0.328,0.048,0.099,...,31.0,67.0,612.0,290.0,0.0,7.0,97.0,532.0,364.0,36798.73
1,0.571,0.543,0.039,0.047,0.071,0.232,0.250,0.329,0.059,0.087,...,28.0,73.0,601.0,298.0,0.0,9.0,82.0,567.0,342.0,46367.63
2,0.583,0.534,0.034,0.035,0.097,0.223,0.264,0.307,0.060,0.097,...,23.0,83.0,576.0,318.0,0.0,6.0,93.0,548.0,353.0,179244.73
3,0.566,0.551,0.039,0.045,0.085,0.198,0.305,0.310,0.049,0.086,...,15.0,72.0,580.0,333.0,0.0,7.0,85.0,524.0,384.0,160322.57
4,0.553,0.565,0.047,0.045,0.084,0.200,0.274,0.300,0.067,0.101,...,27.0,73.0,593.0,307.0,0.0,5.0,98.0,569.0,328.0,250006.63


In [107]:
df_final_train.shape

(330, 25)

In [108]:
df_final_train.columns

Index(['Driver_Total_Female', 'Driver_Total_Male',
       'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_Senior_Ages_65_69',
       'Driver_Total_Upper_Senior_Ages_70_plus', 'Driver_Total_Married',
       'Driver_Total_Single', 'Vehicle_Miles_To_Work', 'Vehicle_Territory',
       'Annual_Premium', 'Vehicle_Anti_Theft_Device_Active Disabling',
       'Vehicle_Anti_Theft_Device_Alarm Only',
       'Vehicle_Anti_Theft_Device_Not Applicable',
       'Vehicle_Anti_Theft_Device_Passive Disabling-Vehicle Recovery',
       'Vehicle_Anti_Theft_Device_Unknown', 'Vehicle_Usage_Business',
       'Vehicle_Usage_Farm    ', 'Vehicle_Usage_Pleasure',
       'Vehicle_Usage_Work    ', 'Loss_Amount'],
      dtype='object')

In [109]:
df_final_train.to_csv('cleaned.csv')

### Training the model with Decision Tree

In [110]:
X_train = df_final_train.loc[:, df_final_train.columns != 'Loss_Amount']
y_train = df_final_train['Loss_Amount']

In [111]:
X_train.shape, y_train.shape

((330, 24), (330,))

In [112]:
X_train.columns

Index(['Driver_Total_Female', 'Driver_Total_Male',
       'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_Senior_Ages_65_69',
       'Driver_Total_Upper_Senior_Ages_70_plus', 'Driver_Total_Married',
       'Driver_Total_Single', 'Vehicle_Miles_To_Work', 'Vehicle_Territory',
       'Annual_Premium', 'Vehicle_Anti_Theft_Device_Active Disabling',
       'Vehicle_Anti_Theft_Device_Alarm Only',
       'Vehicle_Anti_Theft_Device_Not Applicable',
       'Vehicle_Anti_Theft_Device_Passive Disabling-Vehicle Recovery',
       'Vehicle_Anti_Theft_Device_Unknown', 'Vehicle_Usage_Business',
       'Vehicle_Usage_Farm    ', 'Vehicle_Usage_Pleasure',
       'Vehicle_Usage_Work    '],
      dtype='object')

In [113]:
decision_tree_regressor = LinearRegression()
decision_tree_regressor.fit(X_train, y_train)

LinearRegression()

In [114]:
y_train_pred = decision_tree_regressor.predict(X_train)

In [115]:
mae = mean_absolute_error(y_train, y_train_pred)
mae

397214.42814916524

In [116]:
from xgboost import XGBRegressor
xg_regressor = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 3, alpha = 10, n_estimators = 100)
xg_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [117]:
y_train_pred_xg = xg_regressor.predict(np.nan_to_num(X_train.to_numpy()))

In [118]:
mae_xg = mean_absolute_error(y_train, y_train_pred_xg)
mae_xg

515532.4596439394

## Train Using KNN Model

In [119]:
knn_regressor = KNeighborsRegressor(5)
knn_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

KNeighborsRegressor()

In [120]:
y_train_pred_knn = knn_regressor.predict(np.nan_to_num(X_train.to_numpy()))

In [121]:
mae_knn = mean_absolute_error(y_train, y_train_pred_knn)
mae_knn

338249.43784848484

In [122]:
def load_test_portfolios():
    path_testing_dataset = "testing_portfolios/"
    portfolio_files = os.listdir(path_testing_dataset)
    portfolio_dict = {}
    for i in range(len(portfolio_files)):
        portfolio_id = int(portfolio_files[i].split('.')[0][15:])
        portfolio_dict[portfolio_id] = portfolio_files[i]
    od = collections.OrderedDict(sorted(portfolio_dict.items()))
    po_list = []
    for key,val in od.items():
        po_list.append(val)
    df_portfolio_list = []
    for portfolio_file in po_list:
        file_path = os.path.join(path_testing_dataset, portfolio_file)
        df_test_portfolio = pd.read_csv(file_path)
        test_num_features = num_features.copy()
        test_num_features.remove('Loss_Amount')
        
        df_test_num = df_test_portfolio[test_num_features]
        df_test_cat = df_test_portfolio[cat_features]
        df_test_cat_strip = strip_white_spaces(df_test_cat)
        df_test_portfolio = df_test_num.join(df_test_cat_strip, how='outer')
        df_test_portfolio = pd.get_dummies(df_test_portfolio)
        df_test_portfolio = df_test_portfolio.reindex(sorted(X_train.columns), axis=1)
        
        df_portfolio_list.append(df_test_portfolio)
        
    result_dict = collections.OrderedDict()
    for i in range(len(po_list)):
        portfolio_id = po_list[i].split('.')[0][5:]
        result_dict[portfolio_id] = df_portfolio_list[i] 
    return result_dict

In [123]:
def features_to_testing_df(df_portfolios_ordered_dict):
    features_list = mean_columns + sum_columns
    features_list.insert(0, 'ID')
    final_list = []
    for portfolio_id, single_portfolio in df_portfolios_ordered_dict.items():
        ls_portfolio = portfolio_to_features(single_portfolio, False)
        ls_portfolio.insert(0, portfolio_id)
        final_list.append(ls_portfolio)
    df_final_testing = pd.DataFrame(final_list, columns = features_list)
    return df_final_testing

In [124]:
test_portfolios_dict = load_test_portfolios()

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

...

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\sriva\AppData\Local\Temp\ipykernel_35584\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()


In [125]:
for key, value in test_portfolios_dict.items():
    print(key, value.shape)

portfolio_1 (1000, 24)
portfolio_2 (999, 24)
portfolio_3 (1000, 24)
portfolio_4 (997, 24)
portfolio_5 (998, 24)
portfolio_6 (1000, 24)
portfolio_7 (1000, 24)
portfolio_8 (998, 24)
portfolio_9 (998, 24)
portfolio_10 (998, 24)
portfolio_11 (1000, 24)
portfolio_12 (999, 24)
portfolio_13 (999, 24)
portfolio_14 (997, 24)
portfolio_15 (1000, 24)
portfolio_16 (999, 24)
portfolio_17 (998, 24)
portfolio_18 (999, 24)
portfolio_19 (998, 24)
portfolio_20 (997, 24)
portfolio_21 (1000, 24)
portfolio_22 (999, 24)
portfolio_23 (1000, 24)
portfolio_24 (1000, 24)
portfolio_25 (998, 24)
portfolio_26 (1000, 24)
portfolio_27 (998, 24)
portfolio_28 (999, 24)
portfolio_29 (998, 24)
portfolio_30 (998, 24)
portfolio_41 (1000, 24)
portfolio_42 (1000, 24)
portfolio_43 (998, 24)
portfolio_44 (999, 24)
portfolio_45 (999, 24)
portfolio_46 (999, 24)
portfolio_47 (998, 24)
portfolio_48 (1000, 24)
portfolio_49 (999, 24)
portfolio_50 (999, 24)
portfolio_51 (999, 24)
portfolio_52 (998, 24)
portfolio_53 (996, 24)
portfol

In [126]:
df_test_final = features_to_testing_df(test_portfolios_dict)

In [127]:
def generate_test_results(regressor, X_test, output_dir):
    # for each policy
    result_data = collections.OrderedDict()
    portfolio_ids_df = X_test.iloc[:, 0:1]
    X_test_features = X_test.iloc[:, 1:]
    
    y_pred = regressor.predict(np.nan_to_num(X_test_features.to_numpy()))
    loss_ratio = y_pred/X_test['Annual_Premium']
    loss_ratio_log = np.log(loss_ratio)
    loss_ratio_log_df = loss_ratio_log.to_frame(name='ln_LR')
    result_df = portfolio_ids_df.join(loss_ratio_log_df, how='outer')
    
    result_df.to_csv('result.csv', index=False)

In [128]:
generate_test_results(knn_regressor, df_test_final, '')